# Getting started

In the following it is showed how to setup optimization problems in BoFire and how to use strategies to solve them.

## Setting up the optimization problem

In BoFire, an optimization problem is defined by defining a domain containing input and output features as well as constraints (optional).

### Features

Input features can be continuous, discrete, categorical, or categorical with descriptors:

In [ ]:
from bofire.data_models.features.api import ContinuousInput, DiscreteInput, CategoricalInput, CategoricalDescriptorInput

x1 = ContinuousInput(key="x1", bounds=(0,1))
x2 = ContinuousInput(key="x2", bounds=(0,1))
x3 = ContinuousInput(key="x3", bounds=(0,1))
x4 = DiscreteInput(key="x4", values=[1, 2, 5, 7.5])
x5 = CategoricalInput(key="x5", categories=["A", "B", "C"], allowed=[True,True,False])
x6 = CategoricalDescriptorInput(key="x6", categories=["c1", "c2", "c3"], descriptors=["d1", "d2"], values = [[1,2],[2,5],[1,7]])

As output features, currently only continuous output features are supported. Each output feature should have an objective, which can be a minimize or maximize objective. Furthermore, we can define weights between 0 and 1 in case the objectives should not be weighted equally.

In [ ]:
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MaximizeObjective, MinimizeObjective

objective1 = MaximizeObjective(
    w=1.0, 
    lower_bound=0, 
    upper_bound=1,
)
y1 = ContinuousOutput(key="y1", objective=objective1)

objective2 = MinimizeObjective(
    w=1.0
)
y2 = ContinuousOutput(key="y2", objective=objective2)

In- and output features are collected in respective feature lists.

In [ ]:
from bofire.data_models.domain.api import Inputs, Outputs

input_features = Inputs(features = [x1, x2, x3, x4, x5, x6])
output_features = Outputs(features=[y1, y2])

Individual features can be retrieved by name.


In [ ]:
x5 = input_features.get_by_key('x5')
x5

This is also possible with list of feature names.

In [ ]:
input_features.get_by_keys(['x5', 'x2'])

Features of a specific type can be returned by the `get` method, by default it returns all features that are an instance of the provided class.

In [ ]:
input_features.get(CategoricalInput)

By using the `exact` argument one can force it to only return feature of the exact same class.

In [ ]:
input_features.get(CategoricalInput, exact=True)

The `get_keys` method follows the same logic as the `get` method but returns just the keys of the features instead of the features itself.

In [ ]:
input_features.get_keys(CategoricalInput)

The input feature container further provides methods to return a feature container with only all fixed or all free features.

In [ ]:
free_inputs = input_features.get_free()
fixed_inputs = input_features.get_fixed()

One can uniformly sample from individual input features.

In [ ]:
x5.sample(2)

Or directly from input feature containers, uniform, sobol and LHS sampling is possible. A default, uniform sampling is used.

In [ ]:
from bofire.data_models.enum import SamplingMethodEnum

X = input_features.sample(n=10, method=SamplingMethodEnum.LHS)

X

### Constraints

The search space can be further defined by constraints on the input features. BoFire supports linear equality and inequality constraints, as well as non-linear equality and inequality constraints.

#### Linear constraints

`LinearEqualityConstraint` and `LinearInequalityConstraint` are expressions of the form $\sum_i a_i x_i = b$ or $\leq b$ for equality and inequality constraints respectively.
They take a list of names of the input features they are operating on, a list of left-hand-side coefficients $a_i$ and a right-hand-side constant $b$.

In [ ]:
from bofire.data_models.constraints.api import LinearEqualityConstraint, LinearInequalityConstraint

# A mixture: x1 + x2 + x3 = 1
constr1 = LinearEqualityConstraint(features=["x1", "x2", "x3"], coefficients=[1,1,1], rhs=1)

# x1 + 2 * x3 < 0.8
constr2 = LinearInequalityConstraint(features=["x1", "x3"], coefficients=[1, 2], rhs=0.8)

Linear constraints can only operate on `ContinuousInput` features.

#### Nonlinear constraints 

`NonlinearEqualityConstraint` and `NonlinearInequalityConstraint` take any expression that can be evaluated by [pandas.eval](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.eval.html), including mathematical operators such as `sin`, `exp`, `log10` or exponentiation. So far, they cannot be used in any optimizations.

In [ ]:
from bofire.data_models.constraints.api import NonlinearEqualityConstraint, NonlinearInequalityConstraint

# The unit circle: x1**2 + x2**2 = 1
const3 = NonlinearEqualityConstraint(expression="x1**2 + x2**2 - 1")
const3

#### Combinatorial constraint

Use `NChooseKConstraint` to express that we only want to have $k$ out of the $n$ parameters to take positive values.
Think of a mixture, where we have long list of possible ingredients, but want to limit number of ingredients in any given recipe.

In [ ]:
from bofire.data_models.constraints.api import NChooseKConstraint

# Only 2 or 3 out of 3 parameters can be greater than zero
constr5 = NChooseKConstraint(features=["x1", "x2", "x3"], min_count=2, max_count=3, none_also_valid=True)
constr5

Note that we have to set a boolean, if None is also a valid selection, e.g. if we want to have 2 or 3 or none of the ingredients in our recipe.

Similar to the features, constraints can be grouped in a container which acts as the union constraints.

In [ ]:
from bofire.data_models.domain.api import Constraints


constraints = Constraints(constraints=[constr1, constr2])


We can check whether a point satisfies individual constraints or the list of constraints.

In [ ]:
constr2.is_fulfilled(X).values

Output constraints can be setup via sigmoid-shaped objectives passed as argument to the respective feature, which can then also be plotted.


In [ ]:
from bofire.data_models.objectives.api import MinimizeSigmoidObjective
from bofire.plot.api import plot_objective_plotly

output_constraint = MinimizeSigmoidObjective(
    w=1.0, 
    steepness=10,
    tp=0.5
)
y3= ContinuousOutput(key="y3", objective=output_constraint)

output_features = Outputs(features=[y1, y2, y3])

fig = plot_objective_plotly(feature=y3, lower=0, upper=1)

fig.show()

### The domain

The domain holds then all information about an optimization problem and can be understood as a search space defintion.

In [ ]:
from bofire.data_models.domain.api import Domain

domain = Domain(
    inputs=input_features, 
    outputs=output_features, 
    constraints=constraints
    )

In addition one can instantiate the domain also just from lists.

In [ ]:
domain_single_objective = Domain.from_lists(
    inputs=[x1, x2, x3, x4, x5, x6], 
    outputs=[y1], 
    constraints=[]
    )

A summary of the defined features and constraints can be obtained by the methods `get_feature_reps_df()` and `get_constraint_reps_df()`:

In [ ]:
domain.get_feature_reps_df()

In [ ]:
domain.get_constraint_reps_df()

## Optimization

To solve the optimization problem, we further need a solving strategy. BoFire supports strategies without a prediction model such as a random strategy and predictive strategies which are based on a prediction model.

All strategies contain an `ask` method returning a defined number of candidate experiments.

### Random Strategy

In [ ]:
from bofire.data_models.strategies.api import RandomStrategy

import bofire.strategies.mapper as strategy_mapper

strategy_data_model = RandomStrategy(domain=domain)

random_strategy = strategy_mapper.map(strategy_data_model)
random_candidates = random_strategy.ask(2)

random_candidates

### Single objective Bayesian Optimization strategy

Since a predictive strategy includes a prediction model, we need to generate some historical data, which we can afterwards pass as training data to the strategy via the tell method.

For didactic purposes we just choose here from one of our benchmark methods.

In [ ]:
from bofire.benchmarks.single import Himmelblau

benchmark = Himmelblau()

benchmark.domain.get_feature_reps_df()

Generating some initial data works as follows:

In [ ]:
samples = benchmark.domain.inputs.sample(10)

experiments = benchmark.f(samples, return_complete=True)

experiments

Let's setup the SOBO strategy and ask for a candidate.

In [ ]:
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.acquisition_functions.api import qNEI

sobo_strategy_data_model = SoboStrategy(domain=benchmark.domain, acquisition_function=qNEI())

sobo_strategy = strategy_mapper.map(sobo_strategy_data_model)

sobo_strategy.tell(experiments=experiments)

sobo_strategy.ask(candidate_count=1)
